### Creating SparkSession using Jupyter Notebook

In [1]:
import pyspark

myConf=pyspark.SparkConf()

spark = pyspark.sql.SparkSession\
    .builder\
    .master("local")\
    .appName("myApp")\
    .config(conf=myConf)\
    .getOrCreate()

### Data Structure

Spark has 3 structure:\
    **RDD(Resilient Distributed Dataset)**\
        schema free, low-level, unstructured, read-only\
    **DataFrame**\
        structured, schema\
    **DataSet**

### CREATING RDD 

#### creating RDD from list

In [3]:
myList=[1,2,3,4,5,6,7]

In [4]:
myRdd1 = spark.sparkContext.parallelize(myList)

- glom()

In [6]:
#separating list into three rdd using glom
spark.sparkContext.parallelize([1,2,3,4,5,6,7], 3).glom().collect()

[[1, 2], [3, 4], [5, 6, 7]]

#### creating RDD from file

**writing a file into data folder**

%%writefile data/ds_spark_wiki.txt\
 Wikipedia
 Apache Spark is an open source cluster computing framework.
 아파치 스파크는 오픈 소스 클러스터 컴퓨팅 프레임워크이다.
 Apache Spark Apache Spark Apache Spark Apache Spark
 아파치 스파크 아파치 스파크 아파치 스파크 아파치 스파크
 Originally developed at the University of California, Berkeley's AMPLab,
 the Spark codebase was later donated to the Apache Software Foundation,
 which has maintained it since.
 Spark provides an interface for programming entire clusters with
 implicit data parallelism and fault-tolerance.


In [11]:
import os
myRdd2=spark.sparkContext\
    .textFile(os.path.join("data","ds_spark_wiki.txt"))

In [12]:
myRdd2.first()

'Wikipedia'

#### creating RDD from csv file

In [13]:
%%writefile ./data/ds_spark_2cols.csv
35, 2
40, 27
12, 38
15, 31
21, 1
14, 19
46, 1
10, 34
28, 3
48, 1
16, 2
30, 3
32, 2
48, 1
31, 2
22, 1
12, 3
39, 29
19, 37
25, 2

Writing ./data/ds_spark_2cols.csv


In [14]:
myRdd4 = spark.sparkContext\
    .textFile(os.path.join("data","ds_spark_2cols.csv"))

In [15]:
myRdd4.collect()

['35, 2',
 '40, 27',
 '12, 38',
 '15, 31',
 '21, 1',
 '14, 19',
 '46, 1',
 '10, 34',
 '28, 3',
 '48, 1',
 '16, 2',
 '30, 3',
 '32, 2',
 '48, 1',
 '31, 2',
 '22, 1',
 '12, 3',
 '39, 29',
 '19, 37',
 '25, 2']

### Data Transformations

- map(fn)\
요소별로 fn을 적용해서 결과 RDD 돌려줌\
**rdd.map(lambda x: x.split(' '))**

In [16]:
nRdd = spark.sparkContext.parallelize([1, 2, 3, 4])
squared = nRdd.map(lambda x: x * x)
print(squared)
#since the return of map is RDD, to see the actual result .collect() is needed
print(squared.collect())

PythonRDD[9] at RDD at PythonRDD.scala:53
[1, 4, 9, 16]


In [17]:
myRdd5 = myRdd4.map(lambda line: line.split(','))
myRdd5.take(5)

[['35', ' 2'], ['40', ' 27'], ['12', ' 38'], ['15', ' 31'], ['21', ' 1']]

str.split() 함수에 인자가 없으면 whitespace로 분리한다.\
whitespace는 공백이나 탭 등의 기호를 말하는 것으로, 문장을 분리해서 단어로 분리할 경우에 사용한다.

In [ ]:
#문자 개수 세기
myRdd2.map(lambda s:len(s)).collect()

In [ ]:
#대소문자 변환
upperRDD =wordsRdd.map(lambda x: x[0].upper())
print (upperRDD.collect())

foreach vs map

In [18]:
#foreach 반환값X
spark.sparkContext.parallelize([1, 2, 3, 4, 5]).foreach(lambda x: x + 1)

In [19]:
#map 반환값O
spark.sparkContext.parallelize([1, 2, 3, 4, 5]).map(lambda x: x + 1).collect()

[2, 3, 4, 5, 6]

- .filter(fn)\
요소별로 선별하여 fn을 적용해서 결과 RDD 돌려줌\
**rdd.filter(lambda x: "Spark" in x)**

In [ ]:
#"Spark" 단어가 포함된 문장이 조건이된다. count() 함수로 그 갯수를 확인해 보자.

#English spark
myRdd_spark=myRdd2.filter(lambda line: "Spark" in line)
print ("How many lines having 'Spark': ",myRdd_spark.count())
#한글 스파크
myRdd_unicode = myRdd2.filter(lambda line: u"스파크" in line)
print (myRdd_unicode.first())

- .flatMap(fn)\
요소별로 fn을 적용하고, flat해서 결과 RDD 돌려줌\
**rdd.flatMap(lambda x: x.split(' '))**

In [ ]:
#불용어 제거
stopwords = ['is','am','are','the','for','a', 'an', 'at']
myRdd_stop = myRdd2.flatMap(lambda x:x.split())\
                    .filter(lambda x: x not in stopwords)

- groupBy()\
Paired, unpaired RDD에 모두 사용할 수 있지만, 주로 unpaired RDD에 많이 쓰인다.\
paied 는 (key, value) 쌍으로 구성된 데이터, unpaired 는 (key,value)쌍으로 구성되지 않은 데이터.

In [20]:
_testList=[("Seoul",1),("Seoul",1),("Seoul",1),("Busan",1),("Busan",1),
           ("Seoul",1),("Busan",1),
           ("Seoul",1),("Seoul",1),("Busan",1),("Busan",1)]

In [21]:
_testRdd=spark.sparkContext.parallelize(_testList)

In [22]:
_testRdd.groupBy(lambda x:x[0]).collect()

[('Seoul', <pyspark.resultiterable.ResultIterable at 0x1f67d279948>),
 ('Busan', <pyspark.resultiterable.ResultIterable at 0x1f67d279908>)]

groupBy 함수를 적용한 것은 리스트로 변환하여 값을 확인해야한다.\
- mapValues 사용하여 (key1, key2) 형식으로 리스트 만듦\

In [24]:
_testRdd.groupBy(lambda x:x[0]).mapValues(lambda x: list(x)).collect()
#_testRdd.groupBy(lambda x:x[0]).mapValues(list).collect()

[('Seoul',
  [('Seoul', 1),
   ('Seoul', 1),
   ('Seoul', 1),
   ('Seoul', 1),
   ('Seoul', 1),
   ('Seoul', 1)]),
 ('Busan',
  [('Busan', 1), ('Busan', 1), ('Busan', 1), ('Busan', 1), ('Busan', 1)])]

- groupByKey()\
key를 그룹해서 iterator를 돌려줌.\

- pipeline\
함수를 연이어 적용하는 방식\
rdd.map().collect()

#### Saving RDD to file

In [ ]:
spark.sparkContext.parallelize(upper2list).saveAsTextFile("data/ds_spark_wiki_out")

- partition

In [27]:
#partition 늘리기
_testRdd=spark.sparkContext.parallelize(_testList, 2)
#parition 개수 확인
_testRdd.getNumPartitions()

2

In [ ]:
#key민 출력하기
_testRdd.keys().collect()

- Pair RDD\
groupByKey()\
reduceByKey()\
combineByKey()\
aggregateByKey()\
mapValues()

In [26]:
_testList=[("key1",1),("key1",1),("key1",1),("key2",1),("key2",1),
           ("key1",1),("key2",1),
           ("key1",1),("key1",1),("key2",1),("key2",1)]
_testRdd=spark.sparkContext.parallelize(_testList)

### reduceByKey()
동일한 key에 대해 value의 합계를 구한다. (parition 별로 작업 수행)

In [28]:
_testRdd.reduceByKey(lambda x,y:x+y).collect()

[('key1', 6), ('key2', 5)]

### groupByKey()
mapValues를 해야만 결과를 직접적으로 확인가능

In [31]:
_testRdd.groupByKey().collect()

[('key1', <pyspark.resultiterable.ResultIterable at 0x1f67d267408>),
 ('key2', <pyspark.resultiterable.ResultIterable at 0x1f67d2675c8>)]

In [32]:
_testRdd.groupByKey().mapValues(list).collect()

[('key1', [1, 1, 1, 1, 1, 1]), ('key2', [1, 1, 1, 1, 1])]

In [37]:
myRdd2\
    .flatMap(lambda x:x.split())\
    .map(lambda x:(x,1))\
    .groupByKey()\
    .mapValues(sum)\
    .take(10)
#def f(x): return len(x)
#=mapValues(len)

[('Wikipedia', 1),
 ('Apache', 6),
 ('Spark', 7),
 ('is', 1),
 ('an', 2),
 ('open', 1),
 ('source', 1),
 ('cluster', 1),
 ('computing', 1),
 ('framework.', 1)]

### reduceByKey()
reduceByKey()는 groupByKey()와 달리 키별로 빈도를 합산하기 때문에 mapValues()가 필요없다

In [35]:
myRdd2\
    .flatMap(lambda x:x.split())\
    .map(lambda x:(x,1))\
    .reduceByKey(lambda x,y:x+y)\
    .take(10)

[('Wikipedia', 1),
 ('Apache', 6),
 ('Spark', 7),
 ('is', 1),
 ('an', 2),
 ('open', 1),
 ('source', 1),
 ('cluster', 1),
 ('computing', 1),
 ('framework.', 1)]

### countByKey()
reduceByKey() 는 (K,V)로 countByKey()는 dictionary로 병합한다.

In [41]:
myRdd2\
    .flatMap(lambda x:x.split())\
    .map(lambda x:(x,1))\
    .countByKey()
#.items() to be added to get a list

defaultdict(int,
            {'Wikipedia': 1,
             'Apache': 6,
             'Spark': 7,
             'is': 1,
             'an': 2,
             'open': 1,
             'source': 1,
             'cluster': 1,
             'computing': 1,
             'framework.': 1,
             '아파치': 5,
             '스파크는': 1,
             '오픈': 1,
             '소스': 1,
             '클러스터': 1,
             '컴퓨팅': 1,
             '프레임워크이다.': 1,
             '스파크': 4,
             'Originally': 1,
             'developed': 1,
             'at': 1,
             'the': 3,
             'University': 1,
             'of': 1,
             'California,': 1,
             "Berkeley's": 1,
             'AMPLab,': 1,
             'codebase': 1,
             'was': 1,
             'later': 1,
             'donated': 1,
             'to': 1,
             'Software': 1,
             'Foundation,': 1,
             'which': 1,
             'has': 1,
             'maintained': 1,
             'it': 1,
      

### 과제: 단어 빈도 세기 
내림차순으로 정렬

In [ ]:
#불용어 제거
stopwords = set(['및','이를','등','이','이런','그와','또는','두', '이와', '전', '간'])

wc3=myRdd3\
    .flatMap(lambda x:x.split())\
    .filter(lambda x: x.lower() not in stopwords)\
    .map(lambda x:(x,1))\
    .reduceByKey(lambda x,y:x+y)\
    .map(lambda x:(x[1],x[0]))\
    .sortByKey(False)\
    .take(15)

### combineByKey()
(key, (sum, count))\
각 첫 번째 요소에 대해서는 첫 번째 함수(combiner)를 적용하지만,\
똑같은 key가 존재한다면 두 번째 함수(merge value)를 적용하고,\
parition이 여러개라면 세 번째 함수(merge combiners)를 적용해서 merge한다.

In [43]:

_testList=[("key1",1),("key1",3),("key2",2),("key1",2),("key2",4),
           ("key1",5),("key2",6),
           ("key1",7),("key1",8),("key2",9),("key2",3)]

In [44]:
_testRdd=spark.sparkContext.parallelize(_testList)

In [45]:
_testRdd.getNumPartitions()

1

In [46]:
#parition이 1개일 때, 세번째 함수는 작동하지 않음
_testRdd.combineByKey(lambda v : str(v)+"*", lambda c, v : c+"#"+str(v), lambda c1, c2 : c1+'&'+c2).collect()

[('key1', '1*#3#2#5#7#8'), ('key2', '2*#4#6#9#3')]

In [47]:
_testRdd=spark.sparkContext.parallelize(_testList, 2)

partitions = _testRdd.glom().collect()
for num, partition in enumerate(partitions):
    print(f'Partitions {num} -> {partition}')

Partitions 0 -> [('key1', 1), ('key1', 3), ('key2', 2), ('key1', 2), ('key2', 4)]
Partitions 1 -> [('key1', 5), ('key2', 6), ('key1', 7), ('key1', 8), ('key2', 9), ('key2', 3)]


In [49]:
#parition2개 일때, 3번째 함수 적용
_testRdd.combineByKey(lambda v : str(v)+"*", lambda c, v : c+"#"+str(v), lambda c1, c2 : c1+'&'+c2).collect()

[('key1', '1*#3#2&5*#7#8'), ('key2', '2*#4&6*#9#3')]

In [50]:
#각 key에 대해서 value 더하기
_testRdd.combineByKey(lambda value: (value,1),
                     lambda x,value: (x[0]+value, x[1]+1),
                     lambda x,y: (x[0]+y[0], x[1]+y[1])) \
        .collect()

[('key1', (26, 6)), ('key2', (24, 5))]

In [53]:
#각 key 별로 average 구하기
_testCbkRdd=_testRdd.combineByKey(lambda value: (value,1),
                     lambda x,value: (x[0]+value, x[1]+1),                      
                     lambda x,y: (x[0]+y[0], x[1]+y[1]))
averageByKey = _testCbkRdd.map(lambda x:(x[0],x[1][0]/x[1][1]))
averageByKey.collectAsMap()

{'key1': 4.333333333333333, 'key2': 4.8}

In [54]:
#example
marks = spark.sparkContext.parallelize([('kim',86),('lim',87),('kim',75),
                                      ('kim',91),('lim',78),('lim',92),
                                      ('lim',79),('lee',99)])

marksByKey = marks.combineByKey(lambda value: (value,1),
                             lambda x,value: (x[0]+value, x[1]+1),
                             lambda x,y: (x[0]+y[0], x[1]+y[1]))
marksByKey.collect()

[('kim', (252, 3)), ('lim', (336, 4)), ('lee', (99, 1))]

In [55]:
heights = spark.sparkContext.parallelize([
        ('M',182.),('F',164.),('M',180.),('M',185.),('M',171.),('F',162.)
    ])

heightsByKey = heights.combineByKey(lambda value: (value,1),
                             lambda x,value: (x[0]+value, x[1]+1),
                             lambda x,y: (x[0]+y[0], x[1]+y[1]))

heightsByKey.collect()

[('M', (718.0, 4)), ('F', (326.0, 2))]

In [56]:
avgByKey = heightsByKey.map(lambda x: (x[0],x[1][0]/x[1][1]))

print (avgByKey.collectAsMap())

{'M': 179.5, 'F': 163.0}


### 과제: 성적합계 및 평균

#### 답1

In [57]:
marks=[
    "'김하나','English', 100",
    "'김하나','Math', 80",
    "'임하나','English', 70",
    "'임하나','Math', 100",
    "'김갑돌','English', 82.3",
    "'김갑돌','Math', 98.5"
]
_marksRdd=spark.sparkContext.parallelize(marks)

문제 3-1: 이름으로 합계를 구해보자. 올바른 출력은 다음과 같다. 이름과 점수로 데이터를 추출하고, 이름별로 (이름을 키로) 합계를 계산한다.

In [58]:
# marks by name
_marksbyname=_marksRdd\
    .map(lambda x:x.split(','))\
    .map(lambda x: (x[0],float(x[2])))\
    .reduceByKey(lambda x,y:x+y)\
    .collect()

In [59]:
for i in _marksbyname:
  print (i[0],i[1])

'김하나' 180.0
'임하나' 170.0
'김갑돌' 180.8


문제 3-2: 개인별 데이터를 컴마로 분리하고, 과목 x[1]과 성적 x[2]만 꺼내어 reduceBykey()를 구하면 합계를 구할 수 있다.

In [60]:
# marks by subject
_marksbysubject=_marksRdd\
    .map(lambda x:x.split(','))\
    .map(lambda x: (x[1],float(x[2])))\
    .reduceByKey(lambda x,y:x+y)\
    .collect()
for i in _marksbysubject:
  print (i[0],i[1])

'English' 252.3
'Math' 278.5


문제 3-3: 합계, 개수를 계산해 보자. combineByKey()를 이용해서 계산해야 한다. 먼저 데이터를 이름, 과목, 데이터 -> 이름, 점수로 변경한다.

In [61]:
# marks by name
_marksbyname2=_marksRdd\
    .map(lambda x:x.split(','))\
    .map(lambda x: (x[0],float(x[2])))

In [62]:
# sum, counts by name
sum_counts = _marksbyname2.combineByKey(
    (lambda x: (x, 1)), # the initial value, with value x and count 1
    (lambda acc, value: (acc[0]+value, acc[1]+1)), # how to combine a pair value with the accumulator: sum value, and increment count
    (lambda acc1, acc2: (acc1[0]+acc2[0], acc1[1]+acc2[1])) # combine accumulators
)
for i in sum_counts.collect():
    for each in i:
        print (each, end=' ')
    print()

'김하나' (180.0, 2) 
'임하나' (170.0, 2) 
'김갑돌' (180.8, 2) 


문제 3-4: 개인별 평균은 3-3에서 구했던 합계, 개수를 사용하여 계산한다. 평균을 계산하기 위해 float() 형변환을 해주었다

In [63]:
# average
averageByKey = sum_counts\
    .map(lambda x: (x[0],x[1][0]/x[1][1]))\
    .collect()
for i in averageByKey:
    for j in i:
        print (j, end=' ')
    print()

'김하나' 90.0 
'임하나' 85.0 
'김갑돌' 90.4 
